# Тема 6. Сбор данных со сторонних сайтов
*coursera.org*

### Класс для скрапинга

In [1]:
import numpy as np
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import lxml
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pickle
import time

In [2]:

class Scrapping:
       
        
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome('chromedriver.exe')
        
        
    def get_url(self, begin_href, url=None, type_load='requests'):
        
        begin_href = [begin_href] if type(begin_href) == str else begin_href
        url = [self.url] if url==None else url
        
        answer = []
        for u in tqdm_notebook(url):
            soup = self._get_soupe(u, type_load)
            link = soup.findAll('a')
            for l in link:
                temp = l.get('href')
                for bh in begin_href:
                    if temp[:len(bh)]  == bh:
                        answer.append(self.url + temp)  
        return(answer)
    
    
    def _get_soupe(self, url, type_load):
        if type_load == 'requests':
            answer = requests.get(url)
            html = answer.text
        elif type_load == 'webdriver':
            self.driver.get(url)
            
            # так себе вариант ожидания полной загрузки :(
            html=None
            while html != self.driver.page_source:
                html = self.driver.page_source
                time.sleep(5)
            
        return(BeautifulSoup(html, 'lxml'))
    

### Получение ссылок на страницы курсов и специализаций

In [6]:
obj_scr = Scrapping('https://www.coursera.org')
browse = obj_scr.get_url('/browse/')
learn = obj_scr.get_url(begin_href=['/learn/', '/specializations/'], url=browse, type_load = 'webdriver')

In [ ]:
# Сохранение ссылок на страницы в дамп
file = open('learn_link.pcl', 'wb')
pickle.dump(learn, file)
file.close()

### Получение данных со страниц курсов и специализаций

In [7]:
# Востановление ссылок из дампа
file = open('learn_link.pcl', 'rb')
learn = pickle.load(file)
file.close()
len(learn)

891

In [24]:
class ScrappingCoursera:
    
    
    def __init__(self, links):
        self.links = links
        
        chromeOptions = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.images":2} # отключить изображения
        chromeOptions.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chromeOptions)
        self.driver.implicitly_wait(10)
        
        self.driver.get('https://www.coursera.org')
        file = open('cookies.pcl', 'rb')
        cookies = pickle.load(file)
        file.close()
        for cookie in cookies:
            self.driver.add_cookie(cookie)

    
    def _get_page_data(self, url):
        
        self.driver.get(url)
        data = {'link': url}
        
        try: data['head'] = self.driver.find_element_by_class_name('BannerTitle').\
                                        find_element_by_tag_name('h1').text
        except Exception as e: data['head'] = ''
        
        try: data['instructor'] = self._get_instructors()
        except Exception as e: data['instructor'] = ''
        
        try: data['about'] = self.driver.find_element_by_class_name('about-section').\
                                         find_element_by_tag_name('div').text
        except Exception as e: data['about'] = ''
        
        try: data['skills'] = self._get_skills()
        except Exception as e: data['skills'] = ''
        
        try: data['language'] = self.driver.find_element_by_id('Доступные языки').\
                                            find_element_by_xpath('../../..//h4').get_attribute('data-reactid')
        except Exception as e: data['language'] = '!!!'
        
        try: data['duration'] = self.driver.find_element_by_id('Часов на завершение').\
                                            find_element_by_xpath('./../../..//h4').text
        except Exception as e: data['duration'] = '!!!'
        
        return(data)
    
    def _get_instructors(self):
        instructors = self.driver.find_element_by_class_name('Instructors').\
                                  find_elements_by_tag_name('h3')
        temp = []
        for instructor in instructors:
            temp.append(instructor.text)
        return('|'.join(temp))
    
    def _get_skills(self):
        skills = self.driver.find_element_by_class_name('Skills').\
                             find_elements_by_xpath('./div/span')
        temp = []
        for skill in skills:
            temp.append(skill.text)
        return('|'.join(temp))
        

In [25]:
obj = ScrappingCoursera(learn)
data = []
for url in tqdm_notebook(learn[:10]):
    data.append(obj._get_page_data(url))

c:\users\stanislav.lyunchenko\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# Сохранение в дамп
import pickle
file = open('data.pcl', 'wb')
pickle.dump(data, file)
file.close()

In [3]:
# Востановление из дампа
file = open('data.pcl', 'rb')
data = pickle.load(file)
file.close()
len(data)

891

In [26]:
pd.DataFrame(data).head()

,about,duration,head,instructor,language,link,skills
0,This Specialization will develop and enhance y...,,Специализация Become a Journalist: Report the ...,Jeremy Steele|David Poulson|Eric Freedman|Joe ...,201,https://www.coursera.org/specializations/becom...,Journalism|Writing|Professional|News Writing|M...
1,Welcome to Modern Art & Ideas! This course is ...,,Modern Art & Ideas,Lisa Mazzola,248,https://www.coursera.org/learn/modern-art-ideas,Art History|Art|History|Art Direction
2,Graphic Design is all around us! Words and pic...,,Fundamentals of Graphic Design,Michael Worthington,264,https://www.coursera.org/learn/fundamentals-of...,Creativity|Graphics|Design Theory|Color Theory
3,"Graphic design is all around us, in a myriad o...",,Специализация Graphic Design,Michael Worthington|Louise Sandhaus|Anther Kil...,201,https://www.coursera.org/specializations/graph...,Art History|History|Graphics|Graphic Design
4,This Specialization covers elements of three m...,,Специализация Литературное творчество,Brando Skyhorse|Amy Bloom|Amity Gaige|Salvator...,201,https://www.coursera.org/specializations/creat...,Short Story Writing|Fiction Writing|Creativity...
